In [1]:
from operator import itemgetter
from pathlib import Path
import vectorbt as vbt
from dataloader.kraken import load_prices

def load_portfolio(name):
    output_dir = Path.cwd().joinpath('..', 'output')
    filepath = output_dir.joinpath(name)
    return vbt.Portfolio.load(filepath)

def parsedatetime(dt):
    return pd.to_datetime(dt)
    # return datetime.strptime(dt, '%Y-%m-%d %H:%M')

def data_extrapolator(dataset, figure, trace, fields):
    def slicer(start, end):
        start_date = parsedatetime(start)
        end_date = parsedatetime(end)
        delta = end_date - start_date
        delta_m = delta.total_seconds() / 60
        if delta_m < 500:
            interval = 1
        elif delta_m < 500 * 5:
            interval = 5
        elif delta_m < 500 * 15:
            interval = 15
        elif delta_m < 500 * 60:
            interval = 60
        elif delta_m < 500 * 720:
            interval = 720
        else:
            interval = 1440

        fakestart = start_date - timedelta(minutes=interval * 250)
        fakeend = end_date + timedelta(minutes=interval * 250)
        precise_date = dataset[interval].loc[fakestart:fakeend]
        orig_date = dataset[1440]
        the_date = pd.concat([orig_date.query('index < @fakestart or index > @fakeend'), precise_date]).sort_index()
        # f = open('log.txt', "a")
        # f.write(f"[{utils.now()}] {start} {end} {delta} {delta_m} {interval} \n{precise_price} \n {fakestart} {fakeend} \n{the_price.loc[fakestart:fakeend]}\n")
        # f.close()
        with figure.batch_update():
            for key, val in fields.items():
                try:
                    value = getattr(the_date, val)
                except:
                    value = the_date[val]

                setattr(trace, key, value)


    return slicer




In [2]:
pair='XBTUSD'
start_date = "2017-05-15 00:00"
end_date = "2017-05-16 00:00"
interval=1


prices = {}
prices[1] = load_prices(pair, '../data/kraken', start_date, end_date, 1)
prices[5] = load_prices(pair, '../data/kraken', start_date, end_date, 5)
prices[15] = load_prices(pair, '../data/kraken', start_date, end_date, 15)
prices[60] = load_prices(pair, '../data/kraken', start_date, end_date, 60)
prices[720] = load_prices(pair, '../data/kraken', start_date, end_date,720)
prices[1440] = load_prices(pair, '../data/kraken', start_date, end_date, 1440)

In [4]:
from datetime import datetime

def str_to_datetime(str) -> datetime:
    return (
        datetime.strptime(str, "%Y-%m-%d %H:%M:%S")
        if not isinstance(str, datetime)
        else str
    )
orders = [
  # [
  #   "2019-11-25 12:51:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-11-25 13:26:00",
  #   "Direction.SELL",
  #   1.009960131395401
  # ],
  # [
  #   "2019-11-25 13:29:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-11-25 14:31:00",
  #   "Direction.SELL",
  #   0.9981221700355924
  # ],
  # [
  #   "2019-11-25 15:05:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-11-25 15:12:00",
  #   "Direction.SELL",
  #   1.00011758553148
  # ],
  # [
  #   "2019-11-25 15:13:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-11-25 16:19:00",
  #   "Direction.SELL",
  #   0.9996454838164821
  # ],
  # [
  #   "2019-11-25 16:23:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-11-25 16:58:00",
  #   "Direction.SELL",
  #   0.9934648332319124
  # ],
  # [
  #   "2019-11-27 19:16:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-11-27 19:40:00",
  #   "Direction.SELL",
  #   0.9990736054401236
  # ],
  # [
  #   "2019-11-27 19:49:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-11-27 20:37:00",
  #   "Direction.SELL",
  #   1.0081785690570795
  # ],
  # [
  #   "2019-11-27 20:44:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-11-27 22:07:00",
  #   "Direction.SELL",
  #   0.9882652283173878
  # ],
  # [
  #   "2019-12-19 00:28:00",
  #   "Direction.BUY",
  #   None
  # ],
  # [
  #   "2019-12-19 01:01:00",
  #   "Direction.SELL",
  #   0.9934297393743844
  # ]
]

orders = [
    {
      "timestamp":str_to_datetime(order[0]), 
      "order": True if order[1] == "Direction.BUY" else False,
      "price": prices[1].loc[str_to_datetime(order[0])]['open'],
      "profit": order[2]
    } for order in orders
]


In [5]:
import plotly.graph_objects as go
import pandas as pd
import hydra.utils as utils
from datetime import datetime, timedelta


candlestick = go.Candlestick(
        x=prices[1440].index,
        open=prices[1440]["open"],
        high=prices[1440]["high"],
        low=prices[1440]["low"],
        close=prices[1440]["close"],
        hoverinfo="all",
    )

buy_date = [order['timestamp'] for order in orders if order['order']]
sell_date = [order['timestamp'] for order in orders if not order['order']]
buy_price = [order['price'] for order in orders if order['order']]
sell_price = [order['price'] for order in orders if not order['order']]
profit = [order['profit'] for order in orders if not order['order']]

# print(portfolio.total_profit())
# print(portfolio.orders.buy.records)
# buy_idx = portfolio.orders.buy.records['idx']
# buy_date = prices[1].index[buy_idx]
# buy_price = portfolio.orders.buy.records['price']
# buy_size = portfolio.orders.buy.records['size']
# buy_text = [f"{size} | {price}" for size, price in zip(buy_size, buy_price)]

# sell_idx = portfolio.orders.sell.records['idx']
# sell_date = prices[1].index[sell_idx]
# sell_price = portfolio.orders.sell.records['price']
# sell_size = portfolio.orders.sell.records['size']
# sell_text = [f"{size} | {price}" for size, price in zip(sell_size, sell_price)]


buys_trace = go.Scattergl(
    name="Buys",
    marker_color="green",
    x=buy_date,
    y=buy_price,
    mode="markers",
    marker_line_width=2,
    marker_size=10,
)

sells_trace = go.Scattergl(
    name="Sells",
    marker_color="red",
    x=sell_date,
    y=sell_price,
    text = profit,
    mode="markers",
    marker_line_width=2,
    marker_size=10,
    hoverinfo = "x+y+text+name"
)


figure = go.FigureWidget(
    data=[candlestick, buys_trace, sells_trace],
    # data=[candlestick],
    layout=go.Layout(
        title=dict(
            text='FLYBABYFLY'
        ),
        barmode='overlay',
        yaxis={'fixedrange': False},
        yaxis2=dict(
            fixedrange=False,
            domain=[0,1],
        )
    )
)
cs = figure.data[0]



update_candlestick = data_extrapolator(prices, figure, cs, {
    "x": "index",
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close"
})

update_candlestick(start_date,end_date)
def handler(obj, xrange):
    [start, end] = xrange.range
    update_candlestick(start, end)
figure.layout.on_change(handler , 'xaxis')
figure




FigureWidget({
    'data': [{'close': array([1787.85 , 1786.07 , 1788.039, ..., 1742.36 , 1739.75 , 1739.92 ])…

In [6]:
def avg(*arr):
    return sum(arr)/len(arr)

avg(0.2432,
0.2423,
0.2437,
0.2551,
0.2755,
0.3982,
0.3968,
0.4051,
0.4023,
0.4088,)

0.32709999999999995